In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import statsmodels.formula.api as smf

## the deliverable of first part of this notebook is a clean dataset on census tracts: ** df_census**
## uploaded on  https://github.com/picniclin/NYC_yl5240/blob/master/data/civic_census.csv

## Data source is from Geolytics: Neighborhood Change Database (NACD) (http://demographics.geolytics.com/ncdb2010/default.aspx)
the dataset geolytics_nyc_census_1990_2010.csv has been uploaded on:
https://github.com/picniclin/NYC_yl5240/blob/master/data/geolytics_nyc_census_1990_2010.csv


In [2]:
df = pd.read_csv('geolytics_nyc_census_1990_2010.csv')

In [3]:
cols = df.columns.str.replace('TRCTPOP', 'pop')\
            .str.replace('MDFAMY','median_inc_fam_')\
            .str.replace('FAVINC', 'ave_inc_fam_')\
            .str.replace('MDHHY', 'median_inc_hh_')\
            .str.replace('AVHHIN', 'ave_inc_hh_')\
            .str.replace('FALTY', 'fam_inc_')\
            .str.replace('FALT', 'fam_inc_')\
            .str.replace('MXB', 'max_')\
            .str.replace('FAY0', 'fam_inc_')\
            .str.replace('fam_inc_M', 'fam_inc_max_')\
            .str.replace('THY0', 'hh_inc_' )\
            .str.replace('hh_inc_M', 'hh_inc_max_')\
            .str.replace('MDVALHS','median_house_value_')\
            .str.replace('MDGRENT', 'median_rent_' )\
            .str.replace('MCSMORT', 'owner_cost_mo_' )\
            .str.replace('MCSNMOR', 'owner_cost_no_mo_' )\
            .str.replace('OCCHU', 'occ_')\
            .str.replace('VACHU', 'vacant_')\
            .str.replace('RNTOCC','renter_occ_')\
            .str.replace('OWNOCC', 'owner_occ_')\
            .str.replace('SPRNTOC', 'spec_renter_occ_')\
            .str.replace('SPOWNOC', 'spec_owner_occ_')\
            .str.replace('PRSOCU', 'person_occ_')\
            .str.replace('PRSOWNU', 'person_owner_occ_')\
            .str.replace('PRSRNTU', 'person_renter_occ_')\
            .str.replace('POVRAT','poverty')\
            .str.replace('1A', '1')\
            .str.replace('9A', '9')

df.columns = cols

df.rename(columns = {'AREAKEY' : 'census_tract','INTPTLAT':'lat', 'INTPTLON':'lon'}, 
          inplace = True)

In [4]:
#list(df.columns)

In [5]:
df = df.dropna()
df.ZCTA5 = df.ZCTA5.astype('int')

## delete the observation, i.e. census tract, with 0 median rent or 0 median families income
df = df[~((df.median_rent_0 == 0)| (df.median_rent_9 == 0)|(df.median_rent_1 == 0)|\
         (df.median_inc_fam_9 == 0)| (df.median_inc_fam_9 == 0)|(df.median_inc_fam_1 == 0))]


In [6]:
df.head()

,census_tract,lat,lon,ZCTA5,pop9,ave_inc_fam_9,median_inc_fam_9,fam_inc_59,fam_inc_109,fam_inc_139,...,hh_inc_601,hh_inc_751,hh_inc_1001,hh_inc_1251,hh_inc_1501,hh_inc_2001,hh_inc_max_201,median_inc_hh_1,median_house_value_1,median_rent_1
1,36005000200,40.804573,-73.856859,10473,2907,45267.28710,38711,31,29,29,...,83,164,152,203,48,68,11,59826,392500,1273
2,36005000400,40.808915,-73.850488,10473,2260,43694.20068,40484,15,19,24,...,152,190,255,97,161,96,73,71968,393700,1330
3,36005001600,40.818848,-73.858076,10473,5254,30042.84776,24732,61,142,53,...,44,192,171,20,38,20,0,30355,378100,823
4,36005001900,40.800999,-73.909373,10454,1575,23377.47813,24823,76,50,0,...,47,71,14,19,21,0,0,25093,0,870
5,36005002000,40.817580,-73.866096,10473,9110,25062.06250,18762,380,284,133,...,74,114,180,161,0,45,12,24939,398100,455


In [7]:
df.shape

(2089, 108)

In [8]:
df.to_csv('nyc_1990_2010_census.csv')

## NYC has 2089 census tracts in 2010 census.

## Since there is no household income in 1990, we choose family income data as income data

In [9]:
# the tables of different groups of families income level
df_inc_9 = df.loc[:,df.columns[(df.columns == 'census_tract') |\
                (df.columns.str.startswith('fam_inc_')&df.columns.str.endswith('9'))]]

df_inc_0 = df.loc[:,df.columns[(df.columns == 'census_tract') |\
                (df.columns.str.startswith('fam_inc_')&df.columns.str.endswith('0'))]]

df_inc_1 = df.loc[:,df.columns[(df.columns == 'census_tract') |\
               (df.columns.str.startswith('fam_inc_')&df.columns.str.endswith('1'))]]

 
df_inc_9.shape, df_inc_0.shape, df_inc_1.shape

((2089, 20), (2089, 17), (2089, 17))

In [10]:
df_inc_rent = df.loc[:,df.columns[(df.columns == 'census_tract') |\
                                (df.columns.str.startswith('median_inc_fam')) |\
                                (df.columns.str.startswith('median_rent'))]]

In [11]:
df_inc_rent.columns = ['tract', 'inc_9', 'rent_9','inc_0', 'rent_0', 'inc_1',  'rent_1']

df_inc_rent.head()

,tract,inc_9,rent_9,inc_0,rent_0,inc_1,rent_1
1,36005000200,38711,742,50817,730,58368,1273
2,36005000400,40484,535,41019,643,73148,1330
3,36005001600,24732,466,30104,558,33555,823
4,36005001900,24823,422,23671,461,25489,870
5,36005002000,18762,298,19694,266,30817,455


In [12]:
df_inc_rent['ratio_rent_inc_9'] = df_inc_rent.rent_9*12/df_inc_rent.inc_9
df_inc_rent['ratio_rent_inc_0'] = df_inc_rent.rent_0*12/df_inc_rent.inc_0
df_inc_rent['ratio_rent_inc_1'] = df_inc_rent.rent_1*12/df_inc_rent.inc_1
df_inc_rent.head()

,tract,inc_9,rent_9,inc_0,rent_0,inc_1,rent_1,ratio_rent_inc_9,ratio_rent_inc_0,ratio_rent_inc_1
1,36005000200,38711,742,50817,730,58368,1273,0.230012,0.172383,0.261719
2,36005000400,40484,535,41019,643,73148,1330,0.158581,0.188108,0.218188
3,36005001600,24732,466,30104,558,33555,823,0.226104,0.222429,0.294323
4,36005001900,24823,422,23671,461,25489,870,0.204004,0.233704,0.409588
5,36005002000,18762,298,19694,266,30817,455,0.190598,0.162080,0.177175


# 1. Income level groups and entropy index

## 1） Define income level groups
Use median value of the 2089 census tracts' median family income to divide 6 income-level groups:
- < 50%
- [50%, 80%)
- [80%, 100%)
- [100%, 120%)
- [120%, 150%)
- [150%, +inf)


In [13]:
def inc_div(x):
    x1 = x * 0.5
    x2 = x * 0.8
    x3 = x
    x4 = x * 1.2
    x5 = x * 1.5
    return([int(x1), int(x2), int(x3), int(x4), int(x5)])

In [14]:
median_inc_9, median_inc_0, median_inc_1 = df_inc_rent[['inc_9', 'inc_0', 'inc_1']].median()
median_inc_9, median_inc_0, median_inc_1

(36218.0, 42590.0, 56726.0)

In [15]:
## ideal division boundary
inc_div_9 = inc_div(median_inc_9)

inc_div_0 = inc_div(median_inc_0)

inc_div_1 = inc_div(median_inc_1)

In [16]:
inc_div_9, inc_div_0, inc_div_1

([18109, 28974, 36218, 43461, 54327],
 [21295, 34072, 42590, 51108, 63885],
 [28363, 45380, 56726, 68071, 85089])

## calculate the number of families in four income-level groups for each year
the actual division boundary is regarded as the existed income-level boundary in the census data which is nearest to the boudary above

In [17]:
df_inc_9.columns

Index(['census_tract', 'fam_inc_59', 'fam_inc_109', 'fam_inc_139',
       'fam_inc_159', 'fam_inc_189', 'fam_inc_209', 'fam_inc_239',
       'fam_inc_259', 'fam_inc_289', 'fam_inc_309', 'fam_inc_359',
       'fam_inc_409', 'fam_inc_499', 'fam_inc_609', 'fam_inc_759',
       'fam_inc_1009', 'fam_inc_1259', 'fam_inc_1509', 'fam_inc_max_9'],
      dtype='object')

In [18]:
# ideal division boundary for 1990: [18109, 28974, 36218, 43461, 54327]
df_inc_9['inc_group1_9'] = df_inc_9['fam_inc_59'] + df_inc_9['fam_inc_109']\
                        + df_inc_9['fam_inc_139'] + df_inc_9['fam_inc_159'] \
                        + df_inc_9['fam_inc_189'] 
        
df_inc_9['inc_group2_9'] = df_inc_9['fam_inc_209'] + df_inc_9['fam_inc_239'] + \
                           df_inc_9['fam_inc_259'] + df_inc_9['fam_inc_289'] + \
                           df_inc_9['fam_inc_309']
        
df_inc_9['inc_group3_9'] =  df_inc_9['fam_inc_359']

df_inc_9['inc_group4_9'] = df_inc_9['fam_inc_409']

df_inc_9['inc_group5_9'] = df_inc_9['fam_inc_499']  

df_inc_9['inc_group6_9'] =  df_inc_9['fam_inc_609'] + df_inc_9['fam_inc_759'] \
                            + df_inc_9['fam_inc_1009'] + df_inc_9['fam_inc_1259']\
                            + df_inc_9['fam_inc_1509'] + df_inc_9['fam_inc_max_9']

In [19]:
df_inc_div_9 = df_inc_9[['census_tract', 'inc_group1_9', 'inc_group2_9', 
          'inc_group3_9', 'inc_group4_9',
         'inc_group5_9', 'inc_group6_9']]
df_inc_div_9.sum()[1:]

inc_group1_9    452794
inc_group2_9    314860
inc_group3_9    117385
inc_group4_9    106071
inc_group5_9    189509
inc_group6_9    559550
dtype: int64

In [20]:
df_inc_0.columns

Index(['census_tract', 'fam_inc_100', 'fam_inc_150', 'fam_inc_200',
       'fam_inc_250', 'fam_inc_300', 'fam_inc_350', 'fam_inc_400',
       'fam_inc_450', 'fam_inc_500', 'fam_inc_600', 'fam_inc_750',
       'fam_inc_1000', 'fam_inc_1250', 'fam_inc_1500', 'fam_inc_2000',
       'fam_inc_max_200'],
      dtype='object')

In [21]:
# ideal division boundary for 2010: [21295, 34072, 42590, 51108, 63885]
df_inc_0['inc_group1_0'] = df_inc_0['fam_inc_100'] + df_inc_0['fam_inc_150']\
                        + df_inc_0['fam_inc_200'] 
    
df_inc_0['inc_group2_0'] = df_inc_0['fam_inc_250'] + df_inc_0['fam_inc_300'] \
                        + df_inc_0['fam_inc_350']
    
df_inc_0['inc_group3_0'] =  df_inc_0['fam_inc_400']

df_inc_0['inc_group4_0'] =  df_inc_0['fam_inc_450'] + df_inc_0['fam_inc_500']

df_inc_0['inc_group5_0'] = df_inc_0['fam_inc_600'] 

df_inc_0['inc_group6_0'] = df_inc_0['fam_inc_750'] + df_inc_0['fam_inc_1000'] \
                            + df_inc_0['fam_inc_1250'] + df_inc_0['fam_inc_1500']\
                            + df_inc_0['fam_inc_2000']  + df_inc_0['fam_inc_max_200']

In [22]:
df_inc_div_0 = df_inc_0[['census_tract', 'inc_group1_0', 'inc_group2_0', 
          'inc_group3_0', 'inc_group4_0',
         'inc_group5_0', 'inc_group6_0']]
df_inc_div_0.sum()[1:]

inc_group1_0    466241
inc_group2_0    325881
inc_group3_0     97069
inc_group4_0    170127
inc_group5_0    146590
inc_group6_0    646516
dtype: int64

In [23]:
df_inc_1.columns

Index(['census_tract', 'fam_inc_101', 'fam_inc_151', 'fam_inc_201',
       'fam_inc_251', 'fam_inc_301', 'fam_inc_351', 'fam_inc_401',
       'fam_inc_451', 'fam_inc_501', 'fam_inc_601', 'fam_inc_751',
       'fam_inc_1001', 'fam_inc_1251', 'fam_inc_1501', 'fam_inc_2001',
       'fam_inc_max_201'],
      dtype='object')

In [24]:
# ideal division boundary for 2010: [28363, 45380, 56726, 68071, 85089]
df_inc_1['inc_group1_1'] = df_inc_1['fam_inc_101'] + df_inc_1['fam_inc_151']\
                        + df_inc_1['fam_inc_201']  + df_inc_1['fam_inc_251']\
                        + df_inc_1['fam_inc_301']
    
df_inc_1['inc_group2_1'] =  df_inc_1['fam_inc_351'] + df_inc_1['fam_inc_401']\
                        + df_inc_1['fam_inc_451']
    
df_inc_1['inc_group3_1'] =  df_inc_1['fam_inc_501'] + df_inc_1['fam_inc_601']

df_inc_1['inc_group4_1'] =  df_inc_1['fam_inc_751'] 

df_inc_1['inc_group5_1'] =  df_inc_1['fam_inc_1001']

df_inc_1['inc_group6_1'] =  df_inc_1['fam_inc_1251'] + df_inc_1['fam_inc_1501']\
                            + df_inc_1['fam_inc_2001'] + df_inc_1['fam_inc_max_201']
        

In [25]:
df_inc_div_1 = df_inc_1[['census_tract', 'inc_group1_1', 'inc_group2_1', 
          'inc_group3_1', 'inc_group4_1',
         'inc_group5_1', 'inc_group6_1']]
df_inc_div_1.sum()[1:]

inc_group1_1    511033
inc_group2_1    258739
inc_group3_1    208478
inc_group4_1    171592
inc_group5_1    216311
inc_group6_1    473612
dtype: int64

## 2) Calculation of census tracts' entropy index
entropy index of different income-level groups for each census tract from 1990 to 2010
entropy index fomular: https://github.com/picniclin/NYC_yl5240/blob/master/entropy_index_fomular.png

In [26]:
# define entropy index 
def entropy_index(x):
    Q_sum = 0
    for i in x:
        if i == 0:
            Q = 0
        else:
            Q = (-1) * (i/sum(x)) * np.log(i/sum(x))
        Q_sum += Q
    entropy_index = Q_sum/np.log(len(x))
    return entropy_index

In [27]:
df_inc_div_9.is_copy = False
df_inc_div_0.is_copy = False
df_inc_div_1.is_copy = False

In [28]:
# calculate the entropy index 
for i in df_inc_div_9.index:
    df_inc_div_9.loc[i,'entropy_index_9'] = \
            entropy_index(list(df_inc_div_9.loc[i,'inc_group1_9':'inc_group6_9']))

for i in df_inc_div_0.index:
    df_inc_div_0.loc[i,'entropy_index_0'] = \
            entropy_index(list(df_inc_div_0.loc[i,'inc_group1_0':'inc_group6_0']))
        
for i in df_inc_div_1.index:
    df_inc_div_1.loc[i,'entropy_index_1'] = \
            entropy_index(list(df_inc_div_1.loc[i,'inc_group1_1':'inc_group6_1']))

In [29]:
df_inc_div_9.head()

,census_tract,inc_group1_9,inc_group2_9,inc_group3_9,inc_group4_9,inc_group5_9,inc_group6_9,entropy_index_9
1,36005000200,133,177,32,31,84,257,0.867149
2,36005000400,89,101,50,50,89,221,0.919819
3,36005001600,353,381,165,39,161,195,0.908063
4,36005001900,175,42,0,0,67,36,0.652853
5,36005002000,1074,450,131,95,203,303,0.814774


In [30]:
df_inc_div_0.head()

,census_tract,inc_group1_0,inc_group2_0,inc_group3_0,inc_group4_0,inc_group5_0,inc_group6_0,entropy_index_0
1,36005000200,203,94,60,69,72,369,0.857971
2,36005000400,150,191,65,70,84,267,0.923299
3,36005001600,414,303,97,185,92,242,0.925875
4,36005001900,87,90,9,16,30,21,0.830158
5,36005002000,1168,523,133,83,201,217,0.779722


In [31]:
df_inc_div_1.head()

,census_tract,inc_group1_1,inc_group2_1,inc_group3_1,inc_group4_1,inc_group5_1,inc_group6_1,entropy_index_1
1,36005000200,210,240,101,49,136,319,0.921479
2,36005000400,169,157,227,79,206,386,0.943208
3,36005001600,598,301,143,167,129,69,0.862742
4,36005001900,233,61,22,16,12,30,0.672415
5,36005002000,966,427,257,63,103,218,0.804094


In [32]:
df_entropy = df_inc_div_9[['census_tract','entropy_index_9']].merge\
            (df_inc_div_0[['census_tract','entropy_index_0']],on = 'census_tract').merge\
            (df_inc_div_1[['census_tract','entropy_index_1']],on = 'census_tract')

In [33]:
df_entropy.rename(columns = {'census_tract':'tract'}, inplace = True)
df_entropy.head()

,tract,entropy_index_9,entropy_index_0,entropy_index_1
0,36005000200,0.867149,0.857971,0.921479
1,36005000400,0.919819,0.923299,0.943208
2,36005001600,0.908063,0.925875,0.862742
3,36005001900,0.652853,0.830158,0.672415
4,36005002000,0.814774,0.779722,0.804094


In [34]:
df_entropy.shape

(2089, 4)

### Drop abnormal observations with 0 entropy index 

In [35]:
df_entropy[((df_entropy.entropy_index_9 == 0)| (df_entropy.entropy_index_0 == 0)|\
                          (df_entropy.entropy_index_1 == 0))]

,tract,entropy_index_9,entropy_index_0,entropy_index_1
199,36005028400,0.510195,0.515133,0.000000
227,36005033400,0.000000,0.786653,0.695727
616,36047035200,0.191867,0.353255,0.000000
1075,36061000700,0.000000,0.236457,0.438493
1077,36061000900,0.595489,0.167534,0.000000
1101,36061003100,0.000000,0.515210,0.684005
1167,36061010100,0.557739,0.000000,0.595811
1174,36061010900,0.362352,0.377228,0.000000
1185,36061011900,0.610726,0.800328,0.000000
1267,36061019701,0.000000,0.000000,0.461470


In [36]:
df_entropy = df_entropy[~((df_entropy.entropy_index_9 == 0)| (df_entropy.entropy_index_0 == 0)|\
                          (df_entropy.entropy_index_1 == 0))]

df_entropy.shape

(2077, 4)

## 3) Create the dataset for analysis on census tracts

In [37]:
df_census = df_entropy.merge(df_inc_rent, on = 'tract', how = 'left')

In [38]:
df_census.head()

,tract,entropy_index_9,entropy_index_0,entropy_index_1,inc_9,rent_9,inc_0,rent_0,inc_1,rent_1,ratio_rent_inc_9,ratio_rent_inc_0,ratio_rent_inc_1
0,36005000200,0.867149,0.857971,0.921479,38711,742,50817,730,58368,1273,0.230012,0.172383,0.261719
1,36005000400,0.919819,0.923299,0.943208,40484,535,41019,643,73148,1330,0.158581,0.188108,0.218188
2,36005001600,0.908063,0.925875,0.862742,24732,466,30104,558,33555,823,0.226104,0.222429,0.294323
3,36005001900,0.652853,0.830158,0.672415,24823,422,23671,461,25489,870,0.204004,0.233704,0.409588
4,36005002000,0.814774,0.779722,0.804094,18762,298,19694,266,30817,455,0.190598,0.162080,0.177175


In [39]:
df_census.shape

(2077, 13)

In [40]:
df_census.columns

Index(['tract', 'entropy_index_9', 'entropy_index_0', 'entropy_index_1',
       'inc_9', 'rent_9', 'inc_0', 'rent_0', 'inc_1', 'rent_1',
       'ratio_rent_inc_9', 'ratio_rent_inc_0', 'ratio_rent_inc_1'],
      dtype='object')

In [41]:
df_census.to_csv('civic_census.csv')